# Merge Allosteric Site Database with Active Site Information

In [1]:
import gzip
import itertools
import json
import os
from glob import glob

import numpy as np
import pandas as pd
import requests
import tqdm
import xmltodict
from SPARQLWrapper import JSON, SPARQLWrapper

## Input and Output Filenames

In [2]:
asd_xml_dir = '../data/ASD_Release_202306_XF/'    # path to directory containing extracted ASD XML files
asd_csv_file = '../output/ASD_Release_202306.csv'  # file to save the converted CSV file

## Recreate the ASD_Release_202306.txt tab separated table

In [3]:
def parse_allosteric_site(alloteric_site):
    """Convert the list of residues in ASD from "Chain A:HIS25,TYR258; Chain B:VAL325" to ['A-HIS-25', 'A-TYR-258', 'B-VAL-325']"""
    residues = []
    for chain_string in alloteric_site.split('; '):
        chain_name, residue_string = chain_string.split(':')
        chain_id = chain_name[-1]
        for residue in residue_string.split(','):
            res_name, res_id = residue[:3], residue[3:]
            residues.append(f'{chain_id}-{res_name}-{res_id}')
    return residues


def parse_asd_xml(asd_xml_dir):
    data = []
    for xml_file in tqdm.tqdm(glob(os.path.join(asd_xml_dir, '*.xml'))):
        xml_string = open(xml_file, 'r').read()
        xml_string = xml_string.replace('&#x2;', '')  # Remove invalid XML character
        protein = xmltodict.parse(xml_string)
    
        target_gene = ''
        target_id = protein['Organism_Record']['Organism_ID']
    
        if 'Gene' in protein['Organism_Record'] and 'Gene_Name' in protein['Organism_Record']['Gene']:
            target_gene = protein['Organism_Record']['Gene']['Gene_Name']
    
        organism = protein['Organism_Record']['Organism']
    
        # Check if the list of allosteric sites in present in the XML file
        if 'Allosteric_Site_List' in protein['Organism_Record']:
            allosteric_sites = protein['Organism_Record']['Allosteric_Site_List']['Allosteric_Site']
    
            # Check if more than one allosteric site is present
            if isinstance(allosteric_sites, list):
                allosteric_sites = allosteric_sites
            else:
                allosteric_sites = [allosteric_sites]
        else:
            allosteric_sites = []
    
        for site in allosteric_sites:
            if site:
                pdb_uniprot = site['PDB_UniProt_ID'] if 'PDB_UniProt_ID' in site else ''
                allosteric_pdb = site['Allosteric_PDB']
                modulator_serial = site['Modulator_ASD_ID']
                modulator_alias = site['Modulator_Alias']
                modulator_chain = site['Modulator_Chain']
                modulator_class = site['Modulator_Class'] if 'Modulator_Class' in site else ''
                modulator_feature = site['Modulator_Feature'] if 'Modulator_Feature' in site else ''
                modulator_name = site['Modulator_Name'] if 'Modulator_Name' in site else ''
                modulator_resi = site['Modulator_Residue'] if 'Modulator_Residue' in site else ''
                function = site['Function'] if 'Function' in site else ''
                position = site['Position'] if 'Position' in site else ''
                pubmed_id = site['PubMed_ID'] if 'PubMed_ID' in site else ''
                ref_title = site['PubMed_Title'] if 'PubMed_Title' in site else ''
                site_overlap = site['Site_Overlap'] if 'Site_Overlap' in site else ''
                allosteric_site_residue = parse_allosteric_site(site['Allosteric_Site_Residue']) if 'Allosteric_Site_Residue' in site else []
    
                data.append([
                    target_id, target_gene, organism, pdb_uniprot, allosteric_pdb,
                    modulator_serial, modulator_alias, modulator_chain,
                    modulator_class, modulator_feature, modulator_name,
                    modulator_resi, function, position, pubmed_id, ref_title,
                    site_overlap, allosteric_site_residue])
    
    return pd.DataFrame(data, columns=[
        'target_id', 'target_gene', 'organism', 'pdb_uniprot', 'allosteric_pdb',
        'modulator_serial', 'modulator_alias', 'modulator_chain', 'modulator_class',
        'modulator_feature', 'modulator_name', 'modulator_resi', 'function',
        'position', 'pubmed_id', 'ref_title', 'site_overlap',
        'allosteric_site_residues'])

In [4]:
df_asd = parse_asd_xml(asd_xml_dir)
# Make the UniProt and PDB IDs uppercase to facilitate merging on these later
df_asd['allosteric_pdb'] = df_asd['allosteric_pdb'].str.upper()
df_asd['pdb_uniprot'] = df_asd['pdb_uniprot'].str.upper()

100%|██████████| 2419/2419 [00:05<00:00, 455.70it/s]


In [5]:
print('Number of Rows:             ', df_asd.shape[0])
print('Number of Unique PDB IDs:   ', df_asd['allosteric_pdb'].nunique())
print('Number of Unique UniProt AC:', df_asd['pdb_uniprot'].nunique())

Number of Rows:              3172
Number of Unique PDB IDs:    2993
Number of Unique UniProt AC: 701


## Fix Obsolete PDB Entries

The list of obsolete PDB IDs are downloaded from the World Wide Protein Data Bank

While most obsolete PDB IDs have been superseded by only one PDB ID. There exists PDB IDs superseded by multiple PDB IDs. For such cases we select the last PDB ID in the list assuming that it will be the latest one. Obsolete entries without a superseded PDB ID are be removed. 

However, in the ASD dataframe (`df_asd`) each obsolete PDB ID had only one superseded PDB ID. 

In [6]:
response = requests.get('https://files.wwpdb.org/pub/pdb/holdings/all_removed_entries.json.gz')

obsolete_pdb = {}
for pdb_id, value in json.loads(gzip.decompress(response.content)).items():
    if 'superseded_by' in value:
        # Select the last element in the list of superseded PDB IDs
        obsolete_pdb[pdb_id] = value['superseded_by'][-1] 
    else:
        obsolete_pdb[pdb_id] = []

print('Number of Obsolete PDB IDs:', len(obsolete_pdb))

Number of Obsolete PDB IDs: 5917


Obsolete and superseded PDB IDs in ASD

In [7]:
df_asd['allosteric_pdb'] = df_asd['allosteric_pdb'].replace(obsolete_pdb)

In [8]:
print('Number of Rows:             ', df_asd.shape[0])
print('Number of Unique PDB IDs:   ', df_asd['allosteric_pdb'].nunique())
print('Number of Unique UniProt AC:', df_asd['pdb_uniprot'].nunique())

Number of Rows:              3172
Number of Unique PDB IDs:    2991
Number of Unique UniProt AC: 701


## Fix Obsolete UniProt Entries

Get PDB to UniProt Mapping Data

In [9]:
def uniprot_from_pdb(pdb_ids):
    pdb_ids_string = '", "'.join(pdb_ids)

    body = 'query {entries(entry_ids: ["' + pdb_ids_string + '"])' + """
      {
        rcsb_id
        polymer_entities {
          uniprots {
            rcsb_id
          }
        }
      }
    }
    """
    response = requests.post(url='https://data.rcsb.org/graphql', json={"query": body})
    response_data = response.json()

    id_mapping = []
    for record in response_data['data']['entries']:
        rcsb_id = record['rcsb_id']
        uniprot_ids = set()
        for entity in record['polymer_entities']:
            if entity['uniprots']:
                uniprot_ids.add(entity['uniprots'][0]['rcsb_id'])
        id_mapping.append([rcsb_id, uniprot_ids])
        
    return pd.DataFrame(id_mapping, columns=['pdb_id', 'uniprot_id_from_pdb'])


def uniprot_pdb_fix(df):
    df_copy = df[['pdb_id', 'uniprot_id']].drop_duplicates().copy()

    # Get the UniProt IDs of the chains in the proteins from PDB website
    df_pdb = uniprot_from_pdb(df_copy['pdb_id'])
    
    # Count the number of UniProt IDs obtained from PDB website for each PDB ID
    df_pdb['num_uniprot'] = df_pdb['uniprot_id_from_pdb'].apply(len)
    
    df_merged = df_copy.merge(df_pdb, how='left')

    # Check if the UniProt ID in ASD is present in the UniProt IDs obtained from PDB website
    df_merged['present'] = df_merged.apply(lambda x: x['uniprot_id'] in x['uniprot_id_from_pdb'], axis=1)
    df_present = df_merged.loc[df_merged['present'], ['pdb_id', 'uniprot_id']]

    # UniProt IDs absent in the ASD but has only one UniProt ID downloaded from PDB
    df_absent_single = df_merged[~df_merged['present'] & (df_merged['num_uniprot'] == 1)].copy()
    print("PDB IDs Absent from ASD with Single UniProt ID fetched from the PDB")
    display(df_absent_single)

    # Place the only one UniProt ID from the set in Uniprot ID column to pdb_uniprot column
    if not df_absent_single.empty:
        df_absent_single['uniprot_id'] = df_absent_single.apply(lambda x: next(iter(x['uniprot_id_from_pdb'])), axis=1)
    df_absent_single = df_absent_single[['pdb_id', 'uniprot_id']]
    
    df_absent_multiple = df_merged[~df_merged['present'] & (df_merged['num_uniprot'] != 1)]
    print("PDB IDs Absent from ASD with Multiple UniProt ID fetched from the PDB")
    display(df_absent_multiple)

    df_output = pd.concat([df_present, df_absent_single])
    return df_output.sort_values(['pdb_id', 'uniprot_id']).reset_index(drop=True)

def replace_obsolete_pdbs(pdb_set, obsolete_pdbs):
    for pdb in pdb_set & set(obsolete_pdbs.keys()):
        pdb_set.remove(pdb)
        pdb_set.add(obsolete_pdbs[pdb])
        print(f'PDB ID {pdb} has been superseded by {obsolete_pdbs[pdb]}')

In [10]:
df_asd_pdb_uniprot = df_asd[['allosteric_pdb', 'pdb_uniprot']].rename(
    columns={'allosteric_pdb': 'pdb_id', 'pdb_uniprot': 'uniprot_id'}
).drop_duplicates()

df_asd_pdb_uniprot = uniprot_pdb_fix(df_asd_pdb_uniprot)

# Reinsert PDB ID and UniProt AC combination from the manual curation of the table below
df_manual_curation = pd.DataFrame([
    ['1CKK', 'P0DP33'], ['1IQ5', 'P0DP33'], ['1NWD', 'P0DP33'],
    ['3J41', 'P0DP23'], ['3OBK', 'S8F7E9'], ['3RHW', 'G5EBR3'],
    ['3RI5', 'G5EBR3'], ['3RIA', 'G5EBR3'], ['3RIF', 'G5EBR3'],
    ['4A2U', 'P9WP65'], ['4P86', 'P39765'], ['6KDY', 'O43837'],
    ['6UI4', 'I1RCT2'], ['7LD3', 'P30542'], ['7O83', 'P01116']],
    columns=['pdb_id', 'uniprot_id'])

df_asd_pdb_uniprot = pd.concat([df_asd_pdb_uniprot, df_manual_curation])

PDB IDs Absent from ASD with Single UniProt ID fetched from the PDB


,pdb_id,uniprot_id,uniprot_id_from_pdb,num_uniprot,present
197,3MZH,P9WMH2,{P9WMH3},1,False
224,4MBS,P00268,{P51681},1,False
234,4K5Y,P34998,{P00720},1,False
279,4P9D,Q8DSE5,{H6WFU3},1,False
280,4P9D,Q8DSE5,{H6WFU3},1,False
...,...,...,...,...,...
3004,3IWN,,{P09012},1,False
3005,3MUV,,{P09012},1,False
3008,3IRW,,{P09012},1,False
3009,3MXH,,{P09012},1,False


PDB IDs Absent from ASD with Multiple UniProt ID fetched from the PDB


,pdb_id,uniprot_id,uniprot_id_from_pdb,num_uniprot,present
26,7LD3,P30542,"{P04899, P59768, P62873, P08173}",4,False
95,4KH0,E8Y329,{},0,False
96,4KH1,E8Y329,{},0,False
97,4KGX,E8Y329,{},0,False
98,4KGV,E8Y329,{},0,False
156,1NWD,P62158,"{P0DP33, Q07346}",2,False
157,1IQ5,P62158,"{P0DP33, Q3Y416}",2,False
158,3J41,P62158,"{Q6J8I9, P0DP23}",2,False
159,1CKK,P62158,"{P97756, P0DP33}",2,False
198,4A2U,I6Y496,{},0,False


The following PDB IDs from ASD were dropped: 3OWZ, 3OXM, 3OWI, 3Q3Z, 3OXJ, 3OWW, 3OXE because these are structures of glycine riboswitch, an RNA element and do not have an associated UniProt IDs.

In [11]:
df_asd_updated =  df_asd.drop(columns='pdb_uniprot').merge(
    df_asd_pdb_uniprot.rename(columns={'pdb_id': 'allosteric_pdb',
                                       'uniprot_id': 'pdb_uniprot'}),
    on='allosteric_pdb'
)

df_asd_updated['allosteric_site_residues'] = df_asd_updated['allosteric_site_residues'].astype(str)
df_asd_updated.drop_duplicates(inplace=True)

In [12]:
print('Number of Rows:             ', df_asd_updated.shape[0])
print('Number of Unique PDB IDs:   ', df_asd_updated['allosteric_pdb'].nunique())
print('Number of Unique UniProt AC:', df_asd_updated['pdb_uniprot'].nunique())

Number of Rows:              3159
Number of Unique PDB IDs:    2967
Number of Unique UniProt AC: 690


In [13]:
df_asd_updated[['target_id', 'target_gene', 'organism', 'pdb_uniprot', 'allosteric_pdb',
       'modulator_serial', 'modulator_alias', 'modulator_chain',
       'modulator_class', 'modulator_feature', 'modulator_name',
       'modulator_resi', 'function', 'position', 'pubmed_id', 'ref_title',
       'site_overlap', 'allosteric_site_residues'
               ]].to_csv(asd_csv_file, index=False)

## Get Catalytic Site Atlas Data

1. Get the manually curated catalytic residues from the Mechanism and Catalytic Site Atlas (M-CSA)
2. Parse the JSON file and create a pandas dataframe

In [14]:
response = requests.get('https://www.ebi.ac.uk/thornton-srv/m-csa/api/residues/?format=json')
mcsa_data = response.json()

output = []
for i, entry in enumerate(mcsa_data):
    if entry['residue_chains']:
        chain_name          = entry['residue_chains'][0]['chain_name']
        pdb_id              = entry['residue_chains'][0]['pdb_id'].upper()
        assembly_chain_name = entry['residue_chains'][0]['assembly_chain_name']
        assembly            = entry['residue_chains'][0]['assembly']
        code                = entry['residue_chains'][0]['code']
        resid               = entry['residue_chains'][0]['resid']
    else:
        chain_name          = ''
        pdb_id              = ''
        assembly_chain_name = ''
        assembly            = ''
        code                = ''
        resid               = np.nan
    uniprot_id          = entry['residue_sequences'][0]['uniprot_id']
    res_seq_code        = entry['residue_sequences'][0]['code']
    res_seq_resid       = entry['residue_sequences'][0]['resid']

    output.append([chain_name, pdb_id, assembly_chain_name, assembly,
                   code, resid, uniprot_id, res_seq_code, res_seq_resid])

df_csa = pd.DataFrame(output, columns=['chain_name', 'pdb_id',
                                       'assembly_chain_name', 'assembly',
                                       'code', 'resid', 'uniprot_id',
                                       'res_seq_code', 'res_seq_resid'])

In [15]:
# Make the UniProt and PDB IDs uppercase to facilitate merging on these later
df_csa['uniprot_id'] = df_csa['uniprot_id'].str.upper()
df_csa['pdb_id'] = df_csa['pdb_id'].str.upper()

# Replace the assembly_chain_name with chain_name if there are odd chain_name in assembly_chain_name column
df_csa.loc[df_csa['assembly_chain_name'].str.len() > 1, 'assembly_chain_name'] = df_csa['chain_name']
print('Number of UniProt IDs in M-CSA:', df_csa['uniprot_id'].nunique())

Number of UniProt IDs in M-CSA: 989


In [16]:
print('Number of Rows:             ', df_csa.shape[0])
print('Number of Unique PDB IDs:   ', df_csa['pdb_id'].nunique())
print('Number of Unique UniProt AC:', df_csa['uniprot_id'].nunique())

Number of Rows:              5248
Number of Unique PDB IDs:    1003
Number of Unique UniProt AC: 989


3. Download M-CSA PDB IDs

In [17]:
def uniprot_from_pdb_chains(pdb_chains):
    pdb_chains_string = '", "'.join(pdb_chains)
    
    body = 'query {polymer_entity_instances(instance_ids: ["' + pdb_chains_string + '"])' + """
      {
        rcsb_id
        polymer_entity {
          uniprots {
            rcsb_id
          }
        }
      }
    }
    """
    response = requests.post(url='https://data.rcsb.org/graphql', json={"query": body})
    response_data = response.json()

    id_mapping = []
    for record in response_data['data']['polymer_entity_instances']:
        rcsb_id, chain = record['rcsb_id'].split('.')
        if record['polymer_entity']['uniprots'] is not None:
            uniprot_id = record['polymer_entity']['uniprots'][0]['rcsb_id']
        else:
            uniprot_id = ''
        id_mapping.append([rcsb_id, chain, uniprot_id])
        
    return pd.DataFrame(id_mapping, columns=['pdb_id', 'chain', 'uniprot_id'])

### Update obsolete UniProt IDs

In [18]:
csa_pdb_chains = set(df_csa['pdb_id'] + '.' + df_csa['assembly_chain_name'])
df_csa_pdb = uniprot_from_pdb_chains(csa_pdb_chains)
df_csa_pdb = df_csa_pdb.rename(columns={'chain': 'assembly_chain_name', 'uniprot_id': 'UniProt_AC'})
df_csa_merged = df_csa.merge(df_csa_pdb, how='left', on=['pdb_id', 'assembly_chain_name'])
df_csa_merged.loc[(df_csa_merged['uniprot_id'] != df_csa_merged['UniProt_AC']) & ~df_csa_merged['UniProt_AC'].isna(), 'uniprot_id'] = df_csa_merged['UniProt_AC']

In [19]:
df_csa_merged_subset = df_csa_merged[['chain_name', 'pdb_id', 'assembly_chain_name', 'uniprot_id', 'UniProt_AC']].drop_duplicates()
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10):
    display(
        df_csa_merged_subset[df_csa_merged_subset['uniprot_id'] != df_csa_merged_subset['UniProt_AC']]
    )

,chain_name,pdb_id,assembly_chain_name,uniprot_id,UniProt_AC
1698,,,,P00392,NaN
1710,,,,P17854,NaN
1731,,,,P00903,NaN
1971,,,,Q56310,NaN
2183,,,,Q712I6,NaN
2960,,,,P28332,NaN
2981,,,,A9CG74,NaN
3860,,,,Q57764,NaN
4061,,,,,NaN
4073,,,,P0AES7,NaN


In [20]:
df_csa_pdb_uniprot = df_csa_merged_subset[['pdb_id', 'uniprot_id']].drop_duplicates()
print('Number of Rows:             ', df_csa_pdb_uniprot.shape[0])
print('Number of Unique PDB IDs:   ', df_csa_pdb_uniprot['pdb_id'].nunique())
print('Number of Unique UniProt AC:', df_csa_pdb_uniprot['uniprot_id'].nunique())

Number of Rows:              1045
Number of Unique PDB IDs:    1003
Number of Unique UniProt AC: 1037


3. Filter the M-CSA data for proteins contained in the ASD

In [21]:
df_asd_updated = pd.read_csv(asd_csv_file)

In [22]:
df_csa_in_asd = df_csa_merged[df_csa_merged['UniProt_AC'].isin(df_asd_updated['pdb_uniprot'])].copy()

4. Group by PDB ID and UniProt ID

In [23]:
df_csa_in_asd['catalytic_site'] = (df_csa_in_asd['assembly_chain_name'] + '-'
                                   + df_csa_in_asd['res_seq_resid'].astype(str)
                                   + '-' + df_csa_in_asd['res_seq_code']
                                   )

df_csa_in_asd['catalytic_site_resids'] = (
    df_csa_in_asd['res_seq_resid'])

df_csa_in_asd_grouped = df_csa_in_asd[
    ['UniProt_AC', 'catalytic_site', 'catalytic_site_resids']
].groupby(['UniProt_AC']).agg(set).reset_index()

In [24]:
df_csa_in_asd_grouped

,UniProt_AC,catalytic_site,catalytic_site_resids
0,O75164,"{A-288-Ser, A-177-Tyr, A-188-His, A-170-Gly, A...","{288, 170, 177, 276, 188, 190}"
1,P00183,"{A-253-Thr, A-252-Asp, A-360-Gly, A-359-Leu, A...","{358, 359, 360, 187, 252, 253}"
2,P00452,"{A-462-Cys, A-731-Tyr, A-730-Tyr, A-225-Cys, A...","{225, 462, 437, 439, 441, 730, 731}"
3,P00489,"{A-677-Thr, A-569-Lys, A-570-Arg, A-575-Lys, A...","{677, 681, 570, 569, 378, 575}"
4,P00636,"{A-99-Glu, A-281-Glu, A-75-Asp, A-119-Asp, A-6...","{121, 98, 99, 69, 75, 119, 281, 122}"
5,P00720,"{A-20-Asp, A-11-Glu}","{11, 20}"
6,P00864,"{A-138-His, A-713-Arg, A-543-Asp, A-506-Glu, A...","{581, 713, 138, 396, 506, 543}"
7,P00929,"{A-60-Asp, A-49-Glu, A-175-Tyr, A-234-Gly, A-1...","{102, 233, 234, 235, 173, 175, 49, 183, 60}"
8,P00953,"{A-195-Lys, A-111-Lys, A-192-Lys}","{192, 195, 111}"
9,P00968,"{C-303-Arg, C-722-Gly, C-301-Asn, C-715-Arg, C...","{129, 841, 202, 843, 299, 301, 715, 303, 848, ..."


In [25]:
print('Number of Unique UniProt AC:', df_csa_in_asd_grouped['UniProt_AC'].nunique())

Number of Unique UniProt AC: 57


## Download the Active and Binding Site Data from UniProt

In [26]:
def get_uniprot_site_annotations(uniprot_ids):
    # Set the SPARQL endpoint (UniProt)
    sparql = SPARQLWrapper("https://sparql.uniprot.org/sparql")

    output = []
    for uniprot_subset in tqdm.tqdm(itertools.batched(uniprot_ids, 200)):
        uniprot_string = ' '.join([f'uniprotkb:{id}' for id in uniprot_subset])
                 
        # Define the query
        query_string = f"""
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX faldo: <http://biohackathon.org/resource/faldo#>
SELECT DISTINCT ?uniprot_id ?begin ?end ?site ?comment
WHERE
{{
    VALUES ?protein {{ {uniprot_string} }}
    BIND(substr(str(?protein), strlen(str(uniprotkb:))+1) AS ?uniprot_id)
  
	?protein up:annotation ?annotation .
  {{ ?annotation a up:Binding_Site_Annotation }} UNION {{ ?annotation a up:Active_Site_Annotation }} .
    ?annotation rdf:type ?type .
    BIND(substr(str(?type), strlen(str(up:))+1) AS ?site)
    ?annotation up:range ?range .
	?range faldo:begin/faldo:position ?begin .
	?range faldo:end/faldo:position ?end .
    OPTIONAL
    {{
        ?annotation up:ligand ?ligand .
        ?ligand rdfs:comment ?comment .
    }}
}}
"""
        sparql.setQuery(query_string)
    
        # Set the output format as JSON
        sparql.setReturnFormat(JSON)
        
        # Run the SPARQL query and convert to the defined format
        data = sparql.query().convert()

        # Store the query result
        for result in data["results"]["bindings"]:
            output.append({key: value['value'] for key, value in result.items()})
    return pd.DataFrame(output, columns=['uniprot_id', 'site', 'begin', 'end', 'comment'])


def get_uniprot_sequence(uniprot_ids):
    # Set the SPARQL endpoint (UniProt)
    sparql = SPARQLWrapper("https://sparql.uniprot.org/sparql")

    output = []
    for uniprot_subset in tqdm.tqdm(itertools.batched(uniprot_ids, 200)):
        uniprot_string = ' '.join([f'uniprotkb:{id}' for id in uniprot_subset])
                 
        # Define the query
        query_string = f"""
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?uniprot_id ?sequence
WHERE
{{
    VALUES ?protein {{ {uniprot_string} }}
    BIND(substr(str(?protein), strlen(str(uniprotkb:))+1) AS ?uniprot_id)
    ?protein up:sequence ?isoform .
    ?isoform a up:Simple_Sequence ;
			  rdf:value ?sequence .
}}
"""
        sparql.setQuery(query_string)
    
        # Set the output format as JSON
        sparql.setReturnFormat(JSON)
        
        # Run the SPARQL query and convert to the defined format
        data = sparql.query().convert()

        # Store the query result
        for result in data["results"]["bindings"]:
            output.append({key: value['value'] for key, value in result.items()})
    return pd.DataFrame(output, columns=['uniprot_id', 'sequence'])

In [27]:
# uniprot_ids = df_asd_updated['pdb_uniprot'].unique()

# df_uniprot_site_annotations = get_uniprot_site_annotations(uniprot_ids)
# df_uniprot_site_annotations.to_csv('../data/UniProt_Site_Annotations.csv', index=False)

# df_uniprot_sequences = get_uniprot_sequence(uniprot_ids)
# df_uniprot_sequences.to_csv('../data/UniProt_Sequences.csv', index=False)

In [28]:
df_uniprot_site_annotations = pd.read_csv('../data/UniProt_Site_Annotations.csv')
df_uniprot_sequences = pd.read_csv('../data/UniProt_Sequences.csv')

Fix multiple simple sequences returned by UniProt SparQL and use UniProt REST API to fetch the canonical fasta sequence and remove duplicates

In [29]:
print('Number of Unique UniProt AC:', df_uniprot_site_annotations['uniprot_id'].nunique())

Number of Unique UniProt AC: 555


In [30]:
df_duplicated_sequences = df_uniprot_sequences[df_uniprot_sequences.duplicated('uniprot_id', keep=False)]
for uniprot_id in df_duplicated_sequences['uniprot_id'].unique():
    response = requests.get(f'https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta')
    sequence = ''.join(response.text.split('\n')[1:])
    df_uniprot_sequences.loc[df_uniprot_sequences['uniprot_id'] == uniprot_id, 'sequence'] = sequence
df_uniprot_sequences.drop_duplicates(inplace=True)

In [31]:
df_uniprot_site_annotations.fillna('', inplace=True)
df_uniprot_site_annotations[df_uniprot_site_annotations['comment'].str.contains('alloster')].value_counts('comment')

comment
allosteric activator                                                    97
allosteric activator; ligand shared between dimeric partners            33
allosteric inhibitor                                                    27
allosteric effector                                                     15
allosteric activator; ligand shared between two neighboring subunits    11
allosteric effector that controls substrate specificity                 11
allosteric inhibitor; ligand shared between dimeric partners             5
allosteric inhibitor; ligand shared between homodimeric partners         5
Name: count, dtype: int64

In [32]:
# Number of unique UniProt Accesstion Numbers
df_uniprot_site_annotations.loc[df_uniprot_site_annotations['comment'].str.contains('alloster'), 'uniprot_id'].nunique()

42

In [33]:
df_uniprot_site_annotations[['comment']] = df_uniprot_site_annotations[['comment']].fillna('')

df_active_site_annotations = df_uniprot_site_annotations[~df_uniprot_site_annotations['comment'].str.contains('alloster')].copy()

df_active_site_annotations['begin'] = df_active_site_annotations['begin'].astype(int)
df_active_site_annotations['end'] = df_active_site_annotations['end'].astype(int)

# Group the annotation of each UniProt ID in multiple rows 
active_sites = []
for index, group in df_active_site_annotations.groupby(['uniprot_id']):
    active_site_residues = []
    for _, record in group.iterrows():
        # Expand the annotation ranges to list of residue indices
        residues = list(range(record['begin'], record['end'] + 1))
        active_site_residues.extend(residues)
    active_sites.append([index[0], active_site_residues])

df_active_sites = pd.DataFrame(active_sites, columns=['uniprot_id', 'active_site_residues'])

df_active_sites = df_uniprot_sequences.merge(df_active_sites, how='left')
df_active_sites[['active_site_residues']] = df_active_sites[['active_site_residues']].fillna('[]')
df_active_sites.to_csv('../output/Active_Sites_from_UniProt.csv', index=False)

In [34]:
df_uniprot_site_annotations

,uniprot_id,site,begin,end,comment
0,P08581,Binding_Site_Annotation,1084,1092,
1,P17707,Binding_Site_Annotation,7,7,
2,P08581,Binding_Site_Annotation,1110,1110,
3,P17707,Binding_Site_Annotation,223,223,
4,P17707,Binding_Site_Annotation,247,247,
...,...,...,...,...,...
4952,Q91H74,Active_Site_Annotation,1526,1526,
4953,Q91H74,Active_Site_Annotation,1550,1550,
4954,Q91H74,Active_Site_Annotation,1610,1610,
4955,Q6SZW1,Active_Site_Annotation,642,642,


## Combine M-CSA and UniProt

**Note:** Residue indices listed in M-CSA are in agreement with UniProt

In [35]:
df_uniprot = pd.read_csv('../output/Active_Sites_from_UniProt.csv')
df_uniprot['active_site_residues'] = df_uniprot['active_site_residues'].apply(eval)
df_uniprot.rename(columns={'uniprot_id': 'UniProt_AC'}, inplace=True)

In [36]:
df_uniprot_csa = df_uniprot.merge(df_csa_in_asd_grouped, how='left', on='UniProt_AC')

active_site_residues = []

for ind, record in df_uniprot_csa.iterrows():
    if record['catalytic_site_resids'] is not np.nan:
        catalytic_site_resids = record['catalytic_site_resids']
    else:
        catalytic_site_resids = set()
    active_site_residues.append(sorted(set(record['active_site_residues']) | catalytic_site_resids))

df_uniprot_csa['active_site_residues'] = active_site_residues
df_uniprot_csa.rename(columns={'UniProt_AC': 'pdb_uniprot'}, inplace=True)

In [42]:
df_uniprot_csa.loc[(df_uniprot_csa['active_site_residues'].astype(str) != '[]'), 'pdb_uniprot'].nunique()

556

In [38]:
df_asd_with_active_site = df_asd_updated.merge(df_uniprot_csa, how='left', on='pdb_uniprot')
df_asd_with_active_site.dropna(subset='active_site_residues', inplace=True)

# Drop the rows with no active site residues
df_asd_with_active_site = df_asd_with_active_site[df_asd_with_active_site['active_site_residues'].astype(str) != '[]']

# Drop the rows with no allosteric site residues
df_asd_with_active_site = df_asd_with_active_site[df_asd_with_active_site['allosteric_site_residues'].astype(str) != '[]']

Filter the dataframe with at least one allosteric site reidue and one active or binding site residue

In [39]:
df_asd_with_active_site['Number_of_Binding_Site_Residues'] = df_asd_with_active_site['active_site_residues'].apply(len)
df_asd_with_active_site['Number_of_Allosteric_Site_Residues'] = df_asd_with_active_site['allosteric_site_residues'].apply(len)
df_asd_with_active_site = df_asd_with_active_site[
    (df_asd_with_active_site['Number_of_Binding_Site_Residues'] > 0) &
    (df_asd_with_active_site['Number_of_Allosteric_Site_Residues'] > 0) 
]

In [40]:
print('Number of Rows:             ', df_asd_with_active_site.shape[0])
print('Number of Unique PDB IDs:   ', df_asd_with_active_site['allosteric_pdb'].nunique())
print('Number of Unique UniProt AC:', df_asd_with_active_site['pdb_uniprot'].nunique())

Number of Rows:              2270
Number of Unique PDB IDs:    2151
Number of Unique UniProt AC: 433


In [41]:
df_asd_with_active_site.to_csv('../output/ASD_with_Potential_Binding_Site.csv', index=False)